In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
sb.set()
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
clvdata = pd.read_csv("CLVsummary.csv")
clvdata.drop(columns = clvdata.columns[0], inplace = True)
clvdata

In [ ]:
geodata = pd.read_csv("olist_geolocation_dataset.csv")
geodata.rename(columns={"geolocation_zip_code_prefix": "customer_zip_code_prefix"}, inplace = True)
geodata

In [ ]:
customerdata = pd.read_csv("olist_customers_dataset.csv")
customerdata.drop(columns = 'customer_id', inplace = True)
customerdata

In [ ]:
new = pd.merge(customerdata, clvdata, on='customer_unique_id')
new

In [ ]:
len(new.customer_unique_id.unique())

In [ ]:
new1 = new.drop_duplicates(subset=['customer_unique_id'])
new1 = new1.sort_values(by = 'customer_unique_id')
new1 = new1.reset_index()
new1 = new1.drop(columns = 'index')
new1

In [ ]:
new2 = new1[['customer_unique_id','customer_zip_code_prefix','customer_city','customer_state','predicted_clv']]
new2

In [ ]:
new3 = pd.merge(geodata, new2, on='customer_zip_code_prefix')
new3

# Many duplicates as we are unable to derive customer's exact address. So we will just drop all the duplicates as the exact locations of each duplicate is quite near each other.

In [ ]:
new4 = new3.drop_duplicates(subset=['customer_unique_id'])
new4 = new4.sort_values(by = 'customer_unique_id')
new4 = new4.reset_index()
new4 = new4.drop(columns = 'index')
new4

# Loss of 7 entries from 2015 in new2 to 2008 in new4

### Why?

In [ ]:
miss = list(set(new2['customer_unique_id'])-set(new4['customer_unique_id']))
new2.loc[new2['customer_unique_id'] == miss[0]]

In [ ]:
geodata.loc[geodata['customer_zip_code_prefix'] == 72465]

## The zipcodes do not exist in the geodata csv, so we will just ignore these entries


# Now we have the locations of each customer as well as their corresponding CLV. 

# We can now find out which regions in Brazil generate more profit for Olist, so that Olist's marketing and sales team can concentrate on those areas

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import descartes
from datashader.mpl_ext import dsshow, alpha_colormap
import datashader as ds
import datashader.transfer_functions as tf
import matplotlib

In [ ]:
geometry = [Point(xy) for xy in zip(new4.geolocation_lng, new4.geolocation_lat)]

In [ ]:
new4['geometry'] = geometry

In [ ]:
crs = {'init': 'epsg:4326'}
customer_locs = gpd.GeoDataFrame(new4, crs=crs, geometry=geometry)

In [ ]:
customer_locs[['geometry','geolocation_lat','geolocation_lng']].head(10)

In [ ]:
brazilmap = gpd.read_file("./Brazil/bra_admbnda_adm1_ibge_2020.shp")


In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
norm = matplotlib.colors.Normalize(vmin=math.log(customer_locs['predicted_clv'].min(),10), vmax=math.log(customer_locs['predicted_clv'].max()))
brazilmap.plot(ax=ax, facecolor = 'black')
plt.scatter(customer_locs['geolocation_lng'], 
            customer_locs['geolocation_lat'],
            customer_locs['predicted_clv'],
             norm=matplotlib.colors.LogNorm(vmin=customer_locs['predicted_clv'].min(), vmax=customer_locs['predicted_clv'].max()),
            cmap='hot', c = customer_locs['predicted_clv'],
            marker = 'o', alpha=1)
fig, ax = plt.subplots(figsize = (15,1))
cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=matplotlib.cm.hot, norm = norm,
                                orientation='horizontal')

In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
norm = matplotlib.colors.Normalize(vmin=customer_locs['predicted_clv'].min(), vmax=customer_locs['predicted_clv'].max())
brazilmap.plot(ax=ax, facecolor = 'black')
customer_locs.geometry.plot(marker='o',
                            c = customer_locs['predicted_clv'],
                            cmap = 'hot', 
                            markersize=5, 
                            ax=ax)
fig, ax = plt.subplots(figsize = (15,1))
cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=matplotlib.cm.hot, norm = norm,
                                orientation='horizontal')